In [ ]:
pip install peft

In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
pip install accelerate

In [4]:
from transformers import ( VisionEncoderDecoderModel, 
                          ViTFeatureExtractor, 
                          BitsAndBytesConfig,
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments
                         )

from peft import LoraConfig, get_peft_model
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from PIL import Image
import requests
import pandas as pd
import json
import os
import re

2024-06-09 12:31:02.101260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 12:31:02.101373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 12:31:02.248907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
vit = VisionEncoderDecoderModel.from_pretrained("tuman/vit-rugpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("tuman/vit-rugpt2-image-captioning")

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("tuman/vit-rugpt2-image-captioning")

config.json:   0%|          | 0.00/4.93k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.50G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

In [6]:
vit.decoder.config.pad_token_id = 0
vit.decoder.config.bos_token_id = 1
vit.decoder.config.eos_token_id = 2

In [7]:
df = pd.read_csv('/kaggle/input/avito-desc/aaa_data/items.csv')
image_dir = '/kaggle/input/avito-desc/aaa_data/images'

In [10]:
def clean_text(text):
    text = text.replace('\r', ' ').replace('\n', ' ')  
    regex = r'[^a-zA-Zа-яА-Я0-9.,:;!?()\"\'\[\]{}\- ]'
    return re.sub(regex, '', text)

df['description'] = df['description'].apply(clean_text)


def parse_attrs(attrs):
    try:
        return json.loads(attrs.replace("'", '"'))
    except json.JSONDecodeError as e:
        return {}
    
def get_image_paths(image_ids):
    image_ids_list = image_ids.split(', ')
    return [os.path.join(image_dir, f"{img_id}.jpg") for img_id in image_ids_list][0]

def preprocess_data(row):
    title = 'Заголовок: ' + row['title'] + ','
    attrs = ", ".join([f"{k}: {v}" for k, v in row['parsed_attrs'].items()])
    return f"{title} {attrs}, Описание: "

df.drop(columns=['itemid', 'clothes_type'], inplace=True)
df['parsed_attrs'] = df['attrs'].apply(parse_attrs)

attrs_df = pd.json_normalize(df['parsed_attrs'])
df = pd.concat([df, attrs_df], axis=1)
df['Цена'].fillna(df['Цена'].mean(), inplace=True)
df.fillna('', inplace=True)
df['image_paths'] = df['images'].apply(get_image_paths)
df['input_text'] = df.apply(preprocess_data, axis=1)
df = df[['input_text', 'description', 'image_paths']]

/tmp/ipykernel_34/3094951034.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Цена'].fillna(df['Цена'].mean(), inplace=True)


In [14]:
# df['combined_text'] = df['input_text'] + ' ' + df['description']
# df['tokenized_text'] = df['combined_text'].apply(tokenizer_gpt2)
# df['tokenized_text'] = df['tokenized_text'].apply(lambda x: x.input_ids)
# df['token_length'] = df['tokenized_text'].apply(len)
# max_length = df['token_length'].max()
# print("\nДлина самого длинного по токенам текста:", max_length)

# df['tokenized_text'] = df['input_text'].apply(tokenizer_gpt2)
# df['tokenized_text'] = df['tokenized_text'].apply(lambda x: x.input_ids)
# df['token_length'] = df['tokenized_text'].apply(len)
# max_length = df['token_length'].max()
# print("\nДлина самого длинного по токенам инпута:", max_length)

# df['tokenized_text'] = df['description'].apply(tokenizer_gpt2)
# df['tokenized_text'] = df['tokenized_text'].apply(lambda x: x.input_ids)
# df['token_length'] = df['tokenized_text'].apply(len)
# max_length = df['token_length'].max()
# print("\nДлина самого длинного по токенам описания:", max_length)

In [11]:
df['concat'] = df['input_text'] + ' ' + df['description']

In [12]:
df = df.head(10000)

train_df, eval_df = train_test_split(df, test_size=0.05, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [14]:
def tokenize_function(examples):
    concat = examples['concat']
    img = examples['image_paths']
    img = [Image.open(x).convert("RGB") for x in img]
    img =  feature_extractor(img, return_tensors="pt")
    
    concat = [text + tokenizer_gpt2.eos_token for text in concat]
    concat = tokenizer_gpt2(concat, padding='max_length', truncation=True, max_length=350, return_tensors="pt")
    
    return {
        "labels": concat["input_ids"],
        "pixel_values": img["pixel_values"],
    }

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "description", 'image_paths', 'concat'])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "description", 'image_paths', 'concat'])
print(train_dataset.column_names)


Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['__index_level_0__', 'labels', 'pixel_values']


In [15]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules='all-linear',
    lora_dropout=0.1,
)

model = get_peft_model(vit, lora_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    output_dir="./results",
    logging_dir="./logs",
    logging_steps=10000000,
    save_steps=10000000, 
    save_total_limit=2,
    fp16=True,
    save_strategy="no"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1110: UserWarning: fan_in_fan_out is set to True but the target module is `torch.nn.Linear`. Setting fan_in_fan_out to False.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your br

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,No log


TrainOutput(global_step=4750, training_loss=0.7748991570723684, metrics={'train_runtime': 9844.5068, 'train_samples_per_second': 0.965, 'train_steps_per_second': 0.483, 'total_flos': 8.604109354917888e+18, 'train_loss': 0.7748991570723684, 'epoch': 1.0})

In [35]:
promt = train_df.iloc[7].input_text
img = Image.open(train_df.iloc[7].image_paths).convert('RGB')

pix = feature_extractor(img, return_tensors="pt")
text = tokenizer_gpt2(promt, return_tensors="pt")

In [36]:
promt

'Заголовок: Вечернее платье размер 48, Вид одежды: Женская одежда, Вид объявления: Продаю своё, Место сделки: экспериментальный жилой комплекс Мещерское Озеро, Состояние: Хорошее, Цвет: Золотой, Бренд одежды: Другой, Размер. Женская одежда: 48 (L), Предмет одежды: Платья, Описание: '

In [37]:
train_df.iloc[7].description

'Платье вечернее, ткань похожа на трикотажный велюр или бархат, состав ткани 55 полиэстер, 25 хлопок, 20 спандекс, бу состояние хорошее, размер 48 Длина по спине 98 см ОГ - 99 см ОБ 103 см Высылаю'

In [ ]:
res = model.generate(
    pixel_values=pix.pixel_values.cuda(), 
    decoder_input_ids=text.input_ids.cuda(),
    max_length=400,
    eos_token_id=tokenizer_gpt2.eos_token_id,
#     do_sample=True,
#     temperature=0.7
    )

In [ ]:
tokenizer_gpt2.decode(res[0])

In [ ]:
torch.save(vit.state_dict(), './state_dict')

In [17]:
from IPython.display import FileLink

FileLink(r'./state_dict')

/kaggle/working/state_dict